# Pre-Implementation Notes

- Memory indices = [0, 1, 2, ..., 19]
- Batches start at multiples of batch_size[0,5,10,15]
- Shuffle memorie sthen take batch size chunks
- Two distinct networks instead of shared inputs
- Critic evalutes states (not (state, action) pairs)
- Actor decides what to do based on current state
    - Network outputs probabilities(softmax) for a distribution
    - Exploration due to nature of distribution (probabilistic and set up so that each element has finite probability)
- Memory is fixed to length T(20) steps
- Track states, actions, rewards, dones(terminal flags), values, log probs
- Shuffle memories and sample batches (size = 5)
- Perform 4 epochs of updates on each batch
- Update rule for Actor is complex
    - $L^{CPI}(\theta) = \hat{E}_{t}[\frac{\pi_\theta(a_t | s_t)}{\pi_{\theta old}(a_t | s_t)}\hat{A}_t] = \hat{E}_t[r_t(\theta)\hat{A_t}]$
- Based on ratio of new policy to old (can use logs)
- Also takes into account the advantage ($A_t$)
- Define epsilon (~0.2) for clip/min operations
    - $L^{CLIP}(\theta) = \hat{E}_t[min(r_t(\theta)\hat{A}_t), clip(r_t(\theta), 1-\epsilon, 1+\epsilon)\hat{A}_t]$
    - We will be using this for the loss of the actor network
    - Pessimistic lower bound to the loss (smaller loss, smaller gradient, smaller update)
- Advantage at each time step:
    - $\hat{A}_t = \delta_t + (\gamma\lambda)\delta_{t+1} + ... + ... + (\gamma\lambda)^{T-t+1}\delta_{T-1}$, where $\delta_t = r_t + \gamma V(s_{t+1})-V(s_t)$
    - Tells us the benefit of the new state over the old (equal to the sum of $\delta_t$)
        - $\delta_t$ is the reward at a time step + difference in the estimated value of the new and old state
    - $\gamma$ is the output of the critic network 
    - $\lambda$ is a smoothing parameter that helps us lower variance (~0.95)
    - Using nested for loops
- Critic loss:
    - Return = advantage + critic value (from memory)
    - $L_{critic}$ = MSE(return - critic value (from network))
- Total loss is sum of clipped actor and critic:
    - $L_t^{CLIP + VF + S}(\theta) = \hat{E}_t[L_t^{CLIP}(\theta) - c_1L_t^{VF}(\theta) + c_2S[\pi\theta](s_t)] $
    - We are doing gradient *ascent*, not gradient *descent*
    - $c_1$ = 0.5 (for the loss of the critic)

- **Things we won't implement**
    - Coupled actor critic entropy term (S)
    - Can also be used for continuous actions
    - Won't do multi core CPU implementation $\rightarrow$ GPU

- **Data Structures we will need**
    - Class for replay buffers $\rightarrow$ lists
    - Class for actor network, class for critic network
    - Class for agent (ties everything together)
    - Main loop to train and evaluate

Made possible by https://github.com/philtabor/Youtube-Code-Repository/tree/master/ReinforcementLearning/PolicyGradient/PPO/torch

In [3]:
import os
import numpy as np
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical

# PPO Memory 

In [4]:
class PPOMemory:
    def __init__(self, batch_size):
        self.states = []  # states encounted
        self.probs = []  # log probs
        self.vals = []  # value critic calculates
        self.actions = []  # actions we actually took
        self.rewards = []  # rewards received
        self.dones = []  # terminal flags

        self.batch_size = batch_size

    def generate_batches(self):
        # Have a list of intergesr that correspond to indices of memories, and have batch size chuncks of those memories
        # Shuffle up those indices and take batch size chuncks of those indices
        num_states = len(self.states)
        batch_start = np.arange(0, num_states, self.batch_size)
        indices = np.arange(num_states, dtype=np.int64)
        # shuffle those indices to handle stochatic part of the minibatch of stochastic gradienst ascent
        np.random.shuffle(indices)
        # Takes all possible starting points of batches and goes from those indices to i + batch_size
        batches = [indices[i:i+self.batch_size] for i in batch_start]

        return np.array(self.states), np.array(self.actions), np.array(self.probs), np.array(self.vals), np.array(self.rewards), np.array(self.dones), batches

    def store_memory(self, state, action, probs, vals, reward, done):
        self.states.append(state)
        self.probs.append(probs)
        self.actions.append(action)
        self.vals.append(vals)
        self.rewards.append(reward)
        self.dones.append(done)

    def clear_memory(self):
        self.states = []
        self.probs = []
        self.vals = []
        self.actions = []
        self.rewards = []
        self.dones = []

# ActorNetwork

In [5]:
class ActorNetwork(nn.Module):
    def __init__(self, n_actions, input_dims, alpha,
            fc1_dims=256, fc2_dims=256, chkpt_dir='tmp/ppo'):
        super(ActorNetwork, self).__init__()

        self.checkpoint_file = os.path.join(chkpt_dir, 'actor_torch_ppo')
        self.actor = nn.Sequential(
                nn.Linear(*input_dims, fc1_dims), # manually pass in inputs
                nn.ReLU(),
                nn.Linear(fc1_dims, fc2_dims),
                nn.ReLU(),
                nn.Linear(fc2_dims, n_actions),
                nn.Softmax(dim=-1) # SoftMax takes care of the fact that we're dealing with probabilities and they sum to 1
        )

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        # self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.device = T.device('cpu')
        self.to(self.device)

    def forward(self, state):
        dist = self.actor(state)
        dist = Categorical(dist)
        # Calculating series of probabilities that we will use to get our actual actions
        # We can use that to get the log probabilities for the calculation of the ratio, for the two probabilities of our learning function

        return dist

    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))


# CriticNetwork Class

In [6]:
class CriticNetwork(nn.Module):
    def __init__(self, input_dims, alpha, fc1_dims=256, fc2_dims=256,
            chkpt_dir='tmp/ppo'):
        super(CriticNetwork, self).__init__()

        self.checkpoint_file = os.path.join(chkpt_dir, 'critic_torch_ppo')
        self.critic = nn.Sequential(
                nn.Linear(*input_dims, fc1_dims), # manually pass in inputs. 
                nn.ReLU(),
                nn.Linear(fc1_dims, fc2_dims),
                nn.ReLU(),
                nn.Linear(fc2_dims, 1)
        ) # handles batch size automatically

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        # self.device = T.device('cuda:0' xif T.cuda.is_available() else 'cpu')
        self.device = T.device('cpu')
        self.to(self.device)

    def forward(self, state):
        value = self.critic(state)

        return value

    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))


# Agent Class

In [7]:
class Agent:
    def __init__(self, num_actions, input_dims, gamma=0.99, alpha=0.0003, gae_lambda=0.95, policy_clip = 0.2, batch_size=64, N=2048, num_epochs=10): # N is horizon, number of steps before update
        self.gamma = gamma
        self.policy_clip = policy_clip
        self.num_epochs = num_epochs
        self.gae_lambda = gae_lambda
        self.actor = ActorNetwork(num_actions, input_dims, alpha)
        self.critic = CriticNetwork(input_dims, alpha)
        self.memory = PPOMemory(batch_size)

    def remember(self, state, action, probs, vals, reward, done):
        self.memory.store_memory(state, action, probs, vals, reward, done)
    
    def save_models(self):
        print('... saving models ...')
        self.actor.save_checkpoint()
        self.critic.save_checkpoint()
    
    def load_models(self):
        print('... loading models ...')
        self.actor.load_checkpoint()
        self.critic.load_checkpoint()
    
    def choose_action(self, observation):
        # handle choosing an action that takes an observation of the current state of the environment, converts to torch tensor
        state = T.tensor([observation], dtype=T.float).to(self.actor.device)

        dist = self.actor(state)
        value = self.critic(state)
        action = dist.sample()

        probs = T.squeeze(dist.log_prob(action)).item()
        action = T.squeeze(action).item()
        value = T.squeeze(value).item()

        return action, probs, value

    def learn(self):
        # our learning function
        for _ in range(self.num_epochs):
            state_arr, action_arr, old_probs_arr, vals_arr, reward_arr, dones_arr, batches = self.memory.generate_batches()

            values = vals_arr
            # Start calculating advantages
            advantage = np.zeros(len(reward_arr), dtype=np.float32)
            for t in range(len(reward_arr)-1):
                discount = 1
                advantage_time = 0
                for k in range(t, len(reward_arr)-1):
                    # delta_t part is in the paranthesis
                    advantage_time += discount *(reward_arr[k] + self.gamma * values[k+1] * (1-int(dones_arr[k])) - values[k])
                    discount *= self.gamma * self.gae_lambda # takes care of multiplicative factor (gamma lambda) ^ (T-t+1)delta_T-1
                advantage[t] = advantage_time
            advantage = T.tensor(advantage).to(self.actor.device) 

            values = T.tensor(values).to(self.actor.device)
            for batch in batches:
                states = T.tensor(state_arr[batch], dtype=T.float).to(self.actor.device)
                old_probs = T.tensor(old_probs_arr[batch]).to(self.actor.device)
                actions = T.tensor(action_arr[batch]).to(self.actor.device)
                # we have the bottom of the numerator now(pi old)

                # we now need pi new
                dist = self.actor(states)
                critic_value = self.critic(states)
                critic_value = T.squeeze(critic_value)

                new_probs = dist.log_prob(actions)
                prob_ratio = new_probs.exp() / old_probs.exp()
                weighted_probs = advantage[batch] * prob_ratio
                weighted_clipped_probs = T.clamp(prob_ratio, 1-self.policy_clip, 1+self.policy_clip) * advantage[batch]
                actor_loss = -T.min(weighted_probs, weighted_clipped_probs).mean()

                returns = advantage[batch] + values[batch]
                critic_loss = (returns-critic_value) ** 2
                critic_loss = critic_loss.mean()

                total_loss = actor_loss + 0.5*critic_loss
                self.actor.optimizer.zero_grad() # zero the gradient
                self.critic.optimizer.zero_grad() # zero the gradient
                total_loss.backward() # backpropagate total loss
                self.actor.optimizer.step() # upstep optimizer
                self.critic.optimizer.step() # upstep optimizer
        self.memory.clear_memory()